In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import torchvision
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os

device = 'cpu'
if torch.cuda.is_available():
  device = 'cuda'
  torch.cuda.manual_seed_all(1)

/home/elias/anaconda3/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '
/home/elias/anaconda3/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


# Preprocessing

In [5]:
!mkdir specgrams
!mkdir specgrams/TRAIN
!mkdir specgrams/TRAIN/Cough
!mkdir specgrams/TRAIN/No_Cough
!mkdir specgrams/TEST
!mkdir specgrams/TEST/Cough
!mkdir specgrams/TEST/No_Cough

In [6]:
def wave_to_specgram(folder_path):    
    file_list = os.listdir(folder_path)
    dataset = torch.empty(1)
    for file in file_list:
        if file != '.DS_Store':
            waveform, sample_rate = torchaudio.load(folder_path+'/'+file)
            specgram = torchaudio.transforms.Spectrogram()(waveform)
            specgram_resize = torchvision.transforms.Resize((224,224))(specgram)
            plt.figure(frameon=False)
            plt.axis('off')
            plt.imshow(specgram_resize.log2()[0,:,:].numpy(), cmap='gray')
            plt.savefig('./specgrams/'+folder_path.split('/')[-2]+'/'+folder_path.split('/')[-1]+'/'+file.strip('.wav')+'.png')
            plt.close()

In [8]:
train_cough_dir = './Cough_dataset/Unlabeled_audio/TRAIN/Cough'
train_nocough_dir = './Cough_dataset/Unlabeled_audio/TRAIN/No_Cough'
test_cough_dir = './Cough_dataset/Unlabeled_audio/TEST/Cough'
test_nocough_dir = './Cough_dataset/Unlabeled_audio/TEST/No_Cough'

wave_to_specgram(train_cough_dir)
wave_to_specgram(train_nocough_dir)
wave_to_specgram(test_cough_dir)
wave_to_specgram(test_nocough_dir)

/home/elias/anaconda3/lib/python3.7/site-packages/torch/functional.py:516: UserWarning: stft will require the return_complex parameter be explicitly  specified in a future PyTorch release. Use return_complex=False  to preserve the current behavior or return_complex=True to return  a complex output. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:653.)
  normalized, onesided, return_complex)
/home/elias/anaconda3/lib/python3.7/site-packages/torch/functional.py:516: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:590.)
  normalized, onesided, return_complex)
/home/elias/anaconda3/lib/python3.7/site-packages/matplotlib/image.py:405: UserWarning: Warning: converting a masked element to nan.
  dv = (np.float64(self.norm.vmax) -
/h

In [10]:
train_cough_dir = './Cough_dataset/Unlabeled_audio/TRAIN/Cough'
train_nocough_dir = './Cough_dataset/Unlabeled_audio/TRAIN/No_Cough'
test_cough_dir = './Cough_dataset/Unlabeled_audio/TEST/Cough'
test_nocough_dir = './Cough_dataset/Unlabeled_audio/TEST/No_Cough'
print('total training cough files:', len(os.listdir(train_cough_dir)))
print('total training nocough files:', len(os.listdir(train_nocough_dir)))
print('total validation cough files:', len(os.listdir(test_cough_dir)))
print('total validation nocough files:', len(os.listdir(test_nocough_dir)))

total training cough files: 359
total training nocough files: 301
total validation cough files: 155
total validation nocough files: 130


In [11]:
#train_dataset = ImageFolder(train_dir, transform=transform)
#train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#val_dataset = ImageFolder(validation_dir, transform=transform)
#val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [126]:
## test git

# Building Network

In [55]:
model = nn.Sequential(
    nn.Conv2d(3, 64, 3),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(64, 128, 3),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(128, 256, 3),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(256, 256, 3),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(256*7*7, 512),
    nn.ReLU(),
    nn.Linear(512, 1),
    nn.Sigmoid()
)

model.to(device)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (7): ReLU()
  (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (9): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
  (10): ReLU()
  (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (12): Flatten(start_dim=1, end_dim=-1)
  (13): Linear(in_features=12544, out_features=512, bias=True)
  (14): ReLU()
  (15): Linear(in_features=512, out_features=1, bias=True)
  (16): Sigmoid()
)

In [57]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.BCELoss()